In [1]:
import numpy as np
import os
#from scipy.misc import imread, imresize
import datetime
import os
import cv2
import matplotlib.pyplot as plt

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
#from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In [3]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())
batch_size = 30 #experiment with the batch size

In [4]:
def image_process(image):
    if (image.shape[0] == 120):
        crop_image    = image[:,20:140]
        resized_image = cv2.resize(crop_image, (120,120), interpolation = cv2.INTER_AREA)
    if (image.shape[0] == 360):
        crop_image = image[50:310:,50:310]
        resized_image = cv2.resize(crop_image, (120,120), interpolation = cv2.INTER_AREA)        

    return resized_image

In [5]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = range(30) #create a list of image numbers you want to use for a particular video
    #img_idx = [5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
    i = 1
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size  # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches            
            x = len(img_idx)
            y = 120
            z = 120
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    #we first resize and then crop image
                    image = image_process(image)
                          

                    batch_data[folder,idx,:,:,0] = (image[:,:,0] - image[:,:,0].min())/(image[:,:,0].max() - image[:,:,0].min())  #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - image[:,:,1].min())/(image[:,:,1].max() - image[:,:,1].min())  #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - image[:,:,2].min())/(image[:,:,2].max() - image[:,:,2].min())  #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # Code for the remaining data points which are left after full batches

        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            x = len(img_idx)
            y = 120
            z = 120
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    #we first resize and then crop image
                    image = image_process(image)                                            
                    #normalize image
                    batch_data[folder,idx,:,:,0] = (image[:,:,0] - image[:,:,0].min())/(image[:,:,0].max() - image[:,:,0].min())  #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - image[:,:,1].min())/(image[:,:,1].max() - image[:,:,1].min())  #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - image[:,:,2].min())/(image[:,:,2].max() - image[:,:,2].min())  #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

In [6]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


In [7]:
#Model 2 using CNN+RNN
#import libraries
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation , Dropout
from tensorflow.keras.layers import Conv3D, MaxPooling3D ,MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.layers import TimeDistributed,GRU,Conv2D
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
#resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(120,120,3))

In [ ]:
#for layer in resnet.layers[:-10]:
#    layer.trainable=False

In [ ]:

#cnn = Sequential([resnet])
#cnn.add(Conv2D(64,(3,3)))
#cnn.add(Conv2D(128,(2,2)))
#cnn.add(Flatten())


In [ ]:

#model = Sequential()
#model.add(TimeDistributed(cnn,input_shape=(30,120,120,3)))
#model.add(TimeDistributed(BatchNormalization()))
#model.add(TimeDistributed(MaxPooling2D((2, 2))))
#model.add(TimeDistributed(Flatten()))

#model.add(GRU(128,return_sequences=True))
#model.add(GRU(128))
#model.add(Dropout(0.25))

#model.add(Dense(256,activation='relu'))
#model.add(Dense(256,activation='relu'))
#model.add(Dropout(0.25))

#model.add(Dense(5, activation='softmax'))


In [8]:
from tensorflow.keras.applications import mobilenet
pretrained_mobilenet = mobilenet.MobileNet(weights='imagenet', include_top=False)
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [9]:
model = Sequential()
model.add(TimeDistributed(pretrained_mobilenet,input_shape=(30,120,120,3)))

model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Flatten()))

model.add(GRU(128))
model.add(Dropout(0.25))

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(5, activation='softmax'))

In [10]:
optimiser = optimizers.Adam(lr=0.0001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 30, 3, 3, 1024)    3228864   
_________________________________________________________________
time_distributed_1 (TimeDist (None, 30, 3, 3, 1024)    4096      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 1, 1, 1024)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 1024)          0         
_________________________________________________________________
gru (GRU)                    (None, 128)               443136    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               1

In [11]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [12]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

#checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq='epoch')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)

#LR = reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=3, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [13]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Instructions for updating:
Please use Model.fit, which supports generators.
Source path =  /home/datasets/Project_data/train ; batch size = 30
Epoch 1/30
23/23 [==============================] - ETA: 0s - loss: 1.7198 - categorical_accuracy: 0.2760Source path =  /home/datasets/Project_data/val ; batch size = 30
Batch:  4 Index: 30

Epoch 00001: saving model to model_init_2021-06-2717_35_34.791526/model-00001-1.71983-0.27602-1.37463-0.43000.h5
23/23 [==============================] - 62s 3s/step - loss: 1.7198 - categorical_accuracy: 0.2760 - val_loss: 1.3746 - val_categorical_accuracy: 0.4300
Epoch 2/30
23/23 [==============================] - ETA: 0s - loss: 1.2680 - categorical_accuracy: 0.4928
Epoch 00002: saving model to model_init_2021-06-2717_35_34.791526/model-00002-1.26804-0.49275-1.22278-0.42500.h5
23/23 [==============================] - 11s 469ms/step - loss: 1.2680 - categorical_accuracy: 0.4928 - val_loss: 1.2228 - val_categorical_accuracy: 0.4250
Epoch 3/30
23/23 [=======